In [1]:
using FileIO
using SFRM
using Glob
using Dates
using JamesCore
using Setfield
import Images
import ImageShow
using Unitful, UnitfulEquivalences
using LinearAlgebra
using StaticArrays
using GeometryBasics
using Statistics
using BenchmarkTools
using LeastSquaresOptim
using DataFrames
using XLSX

In [2]:
name = "YEu_AM-17-22"

"YEu_AM-17-22"

In [3]:
ort_coeff = ustrip(uconvert(u"keV", 1.0u"Å^-1", Spectral()))
orients = map(glob("D:/p4p/*$(name)*.p4p")) do file
    p4p = load(file)
    p4p["FILEID"][4], p4p["ORT"] * ort_coeff, basename(file)
end
sort!(unique!(orients), by = first)

2-element Vector{Tuple{DateTime, Array, String}}:
 (DateTime("2025-06-14T16:41:45"), [[-0.6841047476598854 -0.7376222665467596 0.605657378838292; -0.7355145351733952 0.8823224019704305 0.24378745132051555; -0.6082152968361674 -0.23733381342525076 -0.9760397514599959], [-0.24304249226422966 1.1336165137803438 0.1518361575502725; 0.8062353079359903 0.2799084373647313 -0.7992785297650636; -0.8112491793269495 -0.06144218590207944 -0.8398272890974053]], "20250614_YEu_AM-17-22.p4p")
 (DateTime("2025-06-15T09:56:45"), [-0.13571674874041498 -1.1405045179163995 0.22747524708209438; 0.8062405896628435 -0.25732063653999543 -0.8091209905608435; 0.8381358098734407 0.06284993224729762 0.8151642779545539], "20250615_YEu_AM-17-22.p4p")

In [4]:
frames = []
frame = nothing
files = glob("D:/sfrm/$(name)/cu_*.sfrm")
sort!(files, by = mtime)
for file in files
    sfrm = try load(file)
    catch
        println("cant load ", file)
        continue
    end
    if sfrm.time < 300 || sfrm.target != "Cu"
        continue
    end
    sfrm.image[sfrm.image .> 100_000] .= median(sfrm.image)
    if isnothing(frame)
        frame = sfrm
        continue
    end
    if frame.angles == sfrm.angles
        frame.time += sfrm.time
        frame.image[:] += sfrm.image[:]
    else
        push!(frames, frame)
        frame = sfrm
    end
end
if isempty(frames) || last(frames) != frame
    push!(frames, frame)
end
frames

6-element Vector{Any}:
 SiemensFrame(Int32[5 506 … 0 0; 91 193 … 0 344; … ; 0 0 … 82 0; 538 590 … 0 263], Dict{String, Any}("CORRECT" => "INTERNAL, s/n: A112169 Done inside firmware", "HDRBLKS" => 15, "PHD" => [0.99, 0.051], "DARK" => "INTERNAL, s/n: A112169 Done inside firmware", "TARGET" => "Cu", "CELL" => [1.0, 1.0, 1.0, 90.0, 90.0, 90.0], "AXES2" => [0.0, 0.0, 0.0, 0.0], "INCREME" => 4.0, "VERSION" => 18, "NCOLS" => [768, 1]…))
 SiemensFrame(Int32[0 465 … 47 0; 26 163 … 0 265; … ; 0 22 … 52 0; 546 482 … 0 268], Dict{String, Any}("CORRECT" => "INTERNAL, s/n: A112169 Done inside firmware", "HDRBLKS" => 15, "PHD" => [0.99, 0.051], "DARK" => "INTERNAL, s/n: A112169 Done inside firmware", "TARGET" => "Cu", "CELL" => [1.0, 1.0, 1.0, 90.0, 90.0, 90.0], "AXES2" => [0.0, 0.0, 0.0, 0.0], "INCREME" => 4.0, "VERSION" => 18, "NCOLS" => [768, 1]…))
 SiemensFrame(Int32[0 466 … 59 0; 69 155 … 0 402; … ; 0 0 … 24 0; 531 571 … 0 153], Dict{String, Any}("CORRECT" => "INTERNAL, s/n: A112169 Done insid

In [5]:
function mean_angles(frame)
    angles = copy(frame.angles)
    angles[frame.axis] += frame.increment / 2
    Tuple(deg2rad.(angles))
end

mean_angles (generic function with 1 method)

In [6]:
λ_MoKα1 = 0.70931715u"Å"
λ_MoKα2 = 0.713607u"Å"
λ_CuKα1 = 1.5405929u"Å"
λ_CuKα2 = 1.5444274u"Å"
XRay(λ::Unitful.Length) = JamesCore.XRay(ustrip(uconvert(u"keV", λ, Spectral())), 0, 0)

XRay (generic function with 1 method)

In [7]:
function center_hkl(frame, crystal;
        center = Vec2d(387, 505),
        px = 0.1353,
        detector = Detector([0, -px, 0], [0, 0, px], [10frame.distance, center[1] * px, -center[2] * px]),
        phi_axis = RotAxis(0, 0, -1),
        chi_axis = RotAxis(-1, 0, 0),
        omega_axis = RotAxis(0, 0, 1),
        tth_axis = RotAxis(0, 0, 1)
)
    @assert frame.axis == 2
    tth, omega, phi, chi = mean_angles(frame)
    crystal = omega_axis(omega) * chi_axis(chi) * phi_axis(phi) * crystal
    detector = tth_axis(tth) * detector
    xray = XRay(λ_CuKα1)
    _, coord = findmax(frame.image[495:515, :])
    p = detector(coord[2], 505)
    n = normalize(p - crystal.p)
    q = norm(xray.k) * n - xray.k
    crystal.UB \ q .|> round .|> Int
end

center_hkl (generic function with 1 method)

In [8]:
function peak_coords(frame, crystal;
        hkl = center_hkl(frame, crystal),
        center = Vec2d(387, 505),
        px = 0.1353,
        detector = Detector([0, -px, 0], [0, 0, px], [10frame.distance, center[1] * px, -center[2] * px]),
        phi_axis = RotAxis(0, 0, -1),
        chi_axis = RotAxis(-1, 0, 0),
        omega_axis = RotAxis(0, 0, 1),
        tth_axis = RotAxis(0, 0, 1)
)
    @assert frame.axis == 2
    tth, omega, phi, chi = mean_angles(frame)
    crystal_scan = chi_axis(chi) * phi_axis(phi) * crystal
    detector = tth_axis(tth) * detector
    map([λ_CuKα1, λ_CuKα2]) do λ
        q = crystal_scan.UB * hkl
        xray = XRay(λ)
        true_omegas = cone_angles(omega_axis, q => xray.k, pi/2 + asin(hypot(q...) / 2hypot(xray.k...)))
        true_omega_n = findmin(true_omegas) do true_omega
            abs(rem2pi(true_omega - omega, RoundNearest))
        end |> last
        true_omega = true_omegas[true_omega_n]
        crystal = omega_axis(true_omega) * crystal_scan
        q = crystal.UB * hkl
        scat = JamesCore.XRay(xray.k + q, crystal.p)
        x, y = intersect_coord(detector, scat)
        Vec2d(x, y)
    end |> Tuple
end

peak_coords (generic function with 1 method)

In [9]:
function box_approx(frame, crystal;
    coords = peak_coords(frame, crystal),
    scale = [6, 3],
    weights = [2, 1]
)
    c = sum(coords .* weights) / sum(weights) .|> round .|> Int
    w = scale * norm(coords[1] - coords[2]) .|> round .|> Int
    x1, y1 = c - w
    x2, y2 = c + w
    CartesianIndex(y1, x1):CartesianIndex(y2, x2)
end

box_approx (generic function with 1 method)

In [10]:
function param_approx(frame, crystal;
    box = box_approx(frame, crystal),
    coords = peak_coords(frame, crystal),
    σ = 1
)
    max_I, max_n = findmax(frame.image[box])
    peak1 = Vec2d(reverse(Tuple(max_n + box[begin] - CartesianIndex(1, 1))))
    peak2 = peak1 + coords[2] - coords[1]
    Float64[median(frame.image), max_I, peak1..., σ, σ, max_I/2, peak2..., σ, σ]
end

param_approx (generic function with 1 method)

In [11]:
function param_approx_restrict(frame, crystal;
    box = box_approx(frame, crystal),
    coords = peak_coords(frame, crystal),
    σ = 1
)
    max_I, max_n = findmax(frame.image[box])
    peak1 = Vec2d(reverse(Tuple(max_n + box[begin] - CartesianIndex(1, 1))))
    peak2 = peak1 + coords[2] - coords[1]
    Float64[median(frame.image), max_I, peak1..., peak2..., σ, σ]
end

param_approx_restrict (generic function with 1 method)

In [12]:
function gauss2d((y, x), (A0, A1, x1, y1, dx1, dy1, A2, x2, y2, dx2, dy2))
    A0 + A1 * exp(-((x-x1)/dx1)^2/2 -((y-y1)/dy1)^2/2) + A2 * exp(-((x-x2)/dx2)^2/2 -((y-y2)/dy2)^2/2)
end

gauss2d (generic function with 1 method)

In [13]:
function gauss2d_restrict((y, x), (A0, A1, x1, y1, x2, y2, dx, dy))
    A0 + A1 * (exp(-((x-x1)/dx)^2/2 -((y-y1)/dy)^2/2) + 0.5 * exp(-((x-x2)/dx)^2/2 -((y-y2)/dy)^2/2))
end

gauss2d_restrict (generic function with 1 method)

In [14]:
function param_refine(frame, crystal;
        p0 = param_approx(frame, crystal),
        box = box_approx(frame, crystal)
)
    resid(p) = begin
        map(box) do I
            gauss2d(I.I, p) - frame.image[I]
        end
    end
    optimize(resid, p0, LevenbergMarquardt()).minimizer
end

param_refine (generic function with 1 method)

In [ ]:
crystal = SingleCrystal(orients[1][2])
frame = frames[1]
display(frame)
display(center_hkl(frame, crystal))
display(peak_coords(frame, crystal))
display(param_refine(frame, crystal))
Images.Gray.(reverse(frame.image, dims=1) / 1e3)

In [15]:
df = DataFrame(frame = eachindex(frames))
subset!(df, :frame => ByRow(n -> begin
            frame = frames[n]
            frame.target == "Cu" && frame.angles[1] != 0 && frame.axis == 2 && abs(frame.increment) > 0.5
        end
        )
)
transform!(df, :frame => ByRow(n -> begin
            frame = frames[n]
            angles = @. round(rad2deg(rem2pi(deg2rad(frame.angles), RoundNearest)), digits = 4)
            orient_m = findfirst(orient -> orient[1] > frame.created, orients)
            if isnothing(orient_m)
                orient_m = lastindex(orients) + 1
            end
            frame.created, orient_m - 1, 10round(frame.distance, digits = 2), angles[1:3]..., frame.increment, frame.time
        end
        ) => [:created, :orient, :distance, :tthD, :omega, :phi, :inc, :time]
)
transform!(df, [:frame, :orient] => ByRow((n, m) -> begin
            frame, orient = frames[n], orients[m]
            center_hkl(frame, SingleCrystal(orient[2]))
        end
        ) => [:h, :k, :l]
)
transform!(df, [:frame, :orient] => ByRow((n, m) -> begin
            frame, orient = frames[n], orients[m]
            coord1, coord2 = peak_coords(frame, SingleCrystal(orient[2]))
            coord1..., coord2...
        end
        ) => [:x1, :y1, :x2, :y2]
)
transform!(df, [:frame, :orient] => ByRow((n, m) -> begin
            frame, orient = frames[n], orients[m]
            box = box_approx(frame, SingleCrystal(orient[2]))
            maximum(frame.image[box])
        end
        ) => :Imax
)
# subset!(df, :Imax => ByRow(I -> I > 3000))
transform!(df, [:frame, :orient] => ByRow((n, m) -> begin
            frame, orient = frames[n], orients[m]
            param_refine(frame, SingleCrystal(orient[2]))
        end
        ) => [:bg, :A1, :x1, :y1, :σx1, :σy1, :A2, :x2, :y2, :σx2, :σy2]
)
sort!(df, :frame)
select!(df, :frame => ByRow(n -> (n, frames[n].filename)) => [:nframe, :frame], :orient => ByRow(m -> (m, orients[m][3])) => [:norient, :orient], Not(:frame, :orient))

Row,nframe,frame,norient,orient,created,distance,tthD,omega,phi,inc,time,h,k,l,x1,y1,x2,y2,Imax,bg,A1,σx1,σy1,A2,σx2,σy2
,Int64,String,Int64,String,DateTime,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Int32,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_01_0001.sfrm,2,20250615_YEu_AM-17-22.p4p,2025-06-15T10:33:26,96.3,-80.0,-94.0,-124.6,4.0,600.0,-6,8,-4,681.643,507.729,687.063,507.724,10331,158.361,10508.2,1.29994,1.00961,5723.79,1.28722,0.965597
2,2,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_02_0001.sfrm,2,20250615_YEu_AM-17-22.p4p,2025-06-15T10:44:06,96.3,-80.0,86.0,-124.6,4.0,600.0,6,-8,4,681.411,499.493,686.746,499.451,15364,176.337,17939.2,1.34265,1.09274,9276.94,1.34541,1.05589
3,3,cu_YEu_AM-17-22_6m84_96f_M80_OmSc4_600s_03_0001.sfrm,2,20250615_YEu_AM-17-22.p4p,2025-06-15T10:56:59,96.3,-79.9,86.0,-124.6,4.0,600.0,6,-8,4,682.828,499.439,688.215,499.441,16078,166.486,17441.9,1.36655,1.09004,9246.85,1.34901,1.07377
4,4,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_01_0001.sfrm,2,20250615_YEu_AM-17-22.p4p,2025-06-15T11:13:57,96.3,126.2,-61.0,-123.7,4.0,600.0,4,-6,-8,689.559,507.591,684.42,507.537,10539,165.121,11498.2,1.2746,1.05492,5747.37,1.31219,0.934262
5,5,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_02_0001.sfrm,2,20250615_YEu_AM-17-22.p4p,2025-06-15T11:24:36,96.3,126.2,119.0,-123.7,4.0,600.0,-4,6,8,689.794,499.65,684.623,499.611,13678,172.54,14428.9,1.32945,1.09658,7520.53,1.35702,1.03763
6,6,cu_YEu_AM-17-22_4m6m8_96f_P126p2_OmSc4_600s_03_0001.sfrm,2,20250615_YEu_AM-17-22.p4p,2025-06-15T11:34:47,96.3,126.1,119.0,-123.7,4.0,600.0,-4,6,8,688.368,499.654,683.187,499.625,13243,164.072,14683.8,1.32823,1.09942,7387.5,1.38118,1.06111


In [16]:
XLSX.openxlsx("$(name).xlsx", mode="w") do xf
    sheet = xf[1]
    XLSX.rename!(sheet, "$(name)")
    XLSX.writetable!(xf[1], df)
end

In [22]:
axis = RotAxis(0, 0, 1)
omega = 0.9
xray = XRay(1, 0, 0)
q = Vec(0.053, 0.806, 0.734)

3-element Vec{3, Float64} with indices SOneTo(3):
 0.053
 0.806
 0.734

In [23]:
@btime begin
    true_omegas = cone_angles(axis, q => xray.k, pi/2 + asin(hypot(q...) / 2hypot(xray.k...)))
        true_omega_n = findmin(true_omegas) do true_omega
            abs(rem2pi(true_omega - omega, RoundNearest))
        end |> last
        true_omega = true_omegas[true_omega_n]
end

  2.278 μs (44 allocations: 1.05 KiB)


0.8948249135572692